## License Plate Detection using Fuzzy Join 

### Start EVA server

We are reusing the start server notebook for launching the EVA server.

In [23]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File '00-start-eva-server.ipynb' already there; not retrieving.


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
nohup eva_server > eva.log 2>&1 &

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Adding the images to EVADB for analysis

In [24]:
cursor.execute('DROP TABLE IF EXISTS MyImages;')
response = cursor.fetch_all()
print(response)

# Download images
!wget -nc https://www.dropbox.com/s/770stddqfl0psog/license.zip

!unzip -n license.zip

cursor.execute('LOAD IMAGE "license/Cars0.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars1.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars2.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars3.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars4.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars5.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars6.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

@status: ResponseStatus.SUCCESS
@batch: 
                                       0
0  Table Successfully dropped: MyImages
@query_time: 0.0652491394430399
File 'license.zip' already there; not retrieving.

Archive:  license.zip


,0
0,Number of loaded IMAGE: 1


### License Plate Recognition

In [25]:
# Download UDFS

!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/ndarray/fuzzy_join.py

!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/ocr_extractor.py

cursor.execute("""DROP UDF OCRExtractor;""")
response = cursor.fetch_all()
response.as_df()

cursor.execute("""DROP UDF FuzzDistance;""")
response = cursor.fetch_all()
response.as_df()

cursor.execute("""CREATE UDF IF NOT EXISTS OCRExtractor
                  INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                  OUTPUT (labels NDARRAY STR(10),
                          bboxes NDARRAY FLOAT32(ANYDIM, 4),
                          scores NDARRAY FLOAT32(ANYDIM))
                  TYPE  OCRExtraction
                  IMPL  'ocr_extractor.py' ;
      """)
response = cursor.fetch_all()
response.as_df()

cursor.execute("""CREATE UDF IF NOT EXISTS FuzzDistance
                    INPUT (Input_Array1 NDARRAY ANYTYPE, Input_Array2 NDARRAY ANYTYPE)
                    OUTPUT (distance FLOAT(32, 7))
                    TYPE NdarrayUDF
                    IMPL "fuzzy_join.py";
                    """)
response = cursor.fetch_all()
response.as_df()

File 'fuzzy_join.py' already there; not retrieving.

File 'ocr_extractor.py' already there; not retrieving.



,0
0,UDF FuzzDistance successfully added to the dat...


In [26]:
cursor.execute("CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));")
response = cursor.fetch_all()
response.as_df()

""


In [27]:
cursor.execute("LOAD CSV 'data.csv' INTO LicensePlateCSV;")
response = cursor.fetch_all()
response.as_df()


,CSV,Number of loaded frames
0,data.csv,5


In [28]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
response = cursor.fetch_all()
response.as_df()

,ocrextractor.labels,ocrextractor.bboxes,ocrextractor.scores
0,[KLo1CN2555],"[[[240, 115], [425, 115], [425, 178], [240, 17...",[0.20029704378116528]
1,"[Coi, ZZm, PGoNMN112]","[[[320, 96], [342, 96], [342, 104], [320, 104]...","[0.2585151841538518, 0.009615490492723363, 0.1..."
2,[PReINLR],"[[[227, 173], [269, 173], [269, 193], [227, 19...",[0.12317288475094909]
3,[DZI7 YXR],"[[[150, 126], [250, 126], [250, 158], [150, 15...",[0.6978633091234704]
4,"[pPuie, BES]","[[[175.83355283634924, 85.05243648347647], [38...","[0.19272780155017014, 0.9999165839414489]"
5,"[Wet, alamy stock photo]","[[[362, 274], [384, 274], [384, 280], [362, 28...","[0.002928912305910474, 0.9390591079980826]"
6,"[802.LIN, MMay, VIRGINIA, 07]","[[[157, 141], [370, 141], [370, 219], [157, 21...","[0.8629838375738593, 0.3385108709335327, 0.999..."


In [29]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('SELECT * FROM LicensePlateCSV;')
response = cursor.fetch_all()
response.as_df()

,licenseplatecsv._row_id,licenseplatecsv.id,licenseplatecsv.label
0,1,1,KLG1CA2555
1,2,2,PGMN112
2,3,3,PRENUP
3,4,4,DZ17YXR
4,5,5,PUI8BES
5,6,1,KLG1CA2555
6,7,2,PGMN112
7,8,3,PRENUP
8,9,4,DZ17YXR
9,10,5,PUI8BES


### Run Fuzzy Join to match Detected License Plate against Local License Plate Database (csv)

In [31]:
cursor.execute("""
   SELECT * FROM MyImages 
       JOIN LATERAL OCRExtractor(data) AS T(a,b,c) 
       JOIN LicensePlateCSV B 
       ON FuzzDistance(T.a, B.label) > 50;
       """)
response = cursor.fetch_all()
response.as_df()

,myimages._row_id,myimages.name,myimages.data,B._row_id,B.id,B.label,T.a,T.b,T.c
0,1,license/Cars0.png,"[[[25, 75, 100], [73, 130, 159], [52, 127, 158...",1,1,KLG1CA2555,[KLo1CN2555],"[[[240, 115], [425, 115], [425, 178], [240, 17...",[0.20029704378116528]
